In [4]:
import numpy as np
from scipy.linalg import solve

# Define the BVP problem setup functions
def A_func(x):
    # A(x) = 0
    return 0 * x

def B_func(x):
    # B(x) = 0
    return 0 * x

def f_func(x):
    # f(x) = sin (pi x)
    return np.sin(np.pi * x)

def exact_solution_func(x):
    # Exact solution for the BVP
    return (1 / (np.pi**2)) * np.sin(np.pi * x)

def create_tridiagonal_matrix(N, h, A, B, j):
    # Create the tridiagonal matrix for the BVP.
    main_diag = 2 / h**2 + B(j)
    sup_diag = -1 / h**2 - A(j) / (2 * h)
    sub_diag = -1 / h**2 + A(j) / (2 * h)
    
    # Create the tridiagonal matrix
    M = np.diag(main_diag) + np.diag(sup_diag[:-1], 1) + np.diag(sub_diag[1:], -1)
    return M, sup_diag, sub_diag

def create_rhs_vector(N, h, f, j, sup_diag, sub_diag):
    # Create the right-hand side vector for the BVP.
    F = f(j)
    F[0] -= sub_diag[0] * 0 # Boundary condition at x=0
    F[-1] -= sup_diag[-1] * 0 # Boundary condition at x=1
    return F

def solve_bvp(N):
    # Solve the BVP using finite difference methods.
    h = 1 / (N + 1)  # Grid spacing
    x = np.linspace(0, 1, N + 2)  # Full grid including boundaries
    j = x[1:-1]  # Interior points only

    # Create tridiagonal matrix
    M, sup_diag, sub_diag = create_tridiagonal_matrix(N, h, A_func, B_func, j)

    # Create right-hand side vector
    F = create_rhs_vector(N, h, f_func, j, sup_diag, sub_diag)

    # Solve the system
    u_interior = solve(M, F)

    # Full solution including boundaries
    u = np.zeros(N + 2)
    u[1:-1] = u_interior
    return x, u

def compute_max_error(N, x, u_numeric, u_exact):
    # Compute the maximum error between the numerical and exact solution.
    max_error = np.max(np.abs(u_numeric - u_exact))
    return max_error

def compute_error_ratios(errors, N_values):
    # Compute the error ratios between consecutive N values.
    print("\nError Ratios (should approach 4):")
    for i in range(1, len(errors)):
        ratio = errors[i-1] / errors[i]
        print(f"N={N_values[i]}/{N_values[i-1]}: {ratio:.2f}")

def main():
    print("***\nUsing Finite Difference Methods to solve the BVP and compute errors.\n***")

    N_values = [10, 20, 40]
    errors = []

    for N in N_values:
        x, u_numeric = solve_bvp(N)
        u_exact = exact_solution_func(x)
        max_error = compute_max_error(N, x, u_numeric, u_exact)
        errors.append(max_error)
        print(f"N={N}, Max Error={max_error:.4e}")

    # Compute error ratios
    compute_error_ratios(errors, N_values)

# Run the main function
main()

***
Using Finite Difference Methods to solve the BVP and compute errors.
***
N=10, Max Error=6.8448e-04
N=20, Max Error=1.8865e-04
N=40, Max Error=4.9552e-05

Error Ratios (should approach 4):
N=20/10: 3.63
N=40/20: 3.81
